In [3]:
# After making changes in seed customer selection, the code is working fine.
# Have paired_ordered logic with time window.



from datetime import datetime
import json
import time
import csv
import os

def process_vrp_file(file_path, result_folder):
    # Load the JSON input from a file
    with open(file_path, 'r') as f:
        data = json.load(f)

    # Extract data from JSON
    distance_matrix = data['distance_matrix']
    time_matrices = data['time_matrices']
    demands = data['demands']
    serve_time = data['serve_time']
    vehicle_capacities = data['vehicle_capacities']
    vehicle_max_weight = data['vehicle_capacities']
    fixed_cost = data['fixed_cost']
    cost_per_km = data['cost_per_km']
    contract_distance = data['contract_distance']
    vehicle_volume = data['vehicle_volume']
    cost_per_minute = data['cost_per_minute']
    variable_cost_per_minute = data['variable_cost_per_minute']
    contract_time = data['contract_time']
    vehicle_average_speed = data['vehicle_average_speed']
    order_volume = data['order_volume']
    vehicle_volumes = data['vehicle_volumes']
    vehicle_contract_code = data['vehicle_contract_code']
    vehicle_max_travel_distance = data['vehicle_max_travel_distance']
    vehicle_max_available_time = data['vehicle_max_available_time']
    print(vehicle_max_available_time)
    order_types = data['order_type']
    depot = data['depot']
    order_id=data['order_id']
    vehicle_max_order=data["vehicle_max_order"]
    order_priority=data["order_priority"]
   
    num_vehicles = data['num_vehicles']

    paired_orders = data['paired_order']
    paired_dict = {pair[0]: pair[1] for pair in paired_orders}

    
    paired_pickup_orders=data['paired_pickup_orders']
    pickup_order={pair[0]:pair[1] for pair in paired_pickup_orders}

    paired_dropoff_orders=data['paired_dropoff_orders']
    pickup_dropoff_orders={pair[0]:pair[1] for pair in paired_dropoff_orders}

    pairing_dropoff=data["pairing_dropoff"]
    pairing_dropoff={pair[0]:pair[1] for pair in pairing_dropoff}

    

    start_time = time.time()


    def parse_time_windows(time_window_list):
        parsed_times = []
        for tw in time_window_list:
            if not isinstance(tw, str):
                print(f"Error: Found non-string value: {tw}")
                continue
            try:
                parsed_time = datetime.fromisoformat(tw)
                parsed_times.append(parsed_time)
            except ValueError as e:
                print(f"Error parsing {tw}: {e}")
        return parsed_times

    # Convert time window lists
    order_time_window_start = parse_time_windows(data["order_time_window_start"])
    order_time_window_end = parse_time_windows(data["order_time_window_end"])
    vehicle_time_window_start = parse_time_windows(data["vehicle_time_window_start"])
    vehicle_time_window_end = parse_time_windows(data["vehicle_time_window_end"])

    # Combine parsed time windows into tuples
    order_time_windows = list(zip(order_time_window_start, order_time_window_end))
    vehicle_time_windows = list(zip(vehicle_time_window_start, vehicle_time_window_end))

    # Check parsed output
    # print("Parsed Vehicle Time Windows:", vehicle_time_windows)
    # print("Parsed Order Time Windows:", order_time_windows)


   


    def select_seed_customers_by_proximity(distance_matrix, num_vehicles):
        depot_index = 0
        distances = [(i, distance_matrix[depot_index][i]) for i in range(1, len(distance_matrix))]
        distances.sort(key=lambda x: x[1])
        return [distances[i][0] for i in range(min(num_vehicles, len(distances)))]


    # def calculate_route_cost(route, vehicle_index, distance_matrix, time_matrix):
    #     total_distance = 0
    #     total_time = 0

    #     if len(route) > 0:  
    #         depot_index = 0
    #         total_distance += distance_matrix[depot_index][route[0]]  
    #         total_time += time_matrix[depot_index][route[0]]

    #     for i in range(len(route) - 1):
    #         total_distance += distance_matrix[route[i]][route[i + 1]]
    #         total_time += time_matrix[route[i]][route[i + 1]]
    #         total_time += serve_time[route[i]]

    #     if len(route) > 0:  
    #         total_distance += distance_matrix[route[-1]][depot_index]  
    #         total_time += time_matrix[route[-1]][depot_index]
    #         total_time += serve_time[route[-1]]

    #     excess_distance = max(0, total_distance - contract_distance[vehicle_index])
    #     variable_distance_cost = total_distance * cost_per_km[vehicle_index]

    #     if excess_distance > 0:
    #         variable_distance_cost += excess_distance * cost_per_km[vehicle_index]

    #     total_cost = (fixed_cost[vehicle_index] +
    #                   variable_distance_cost +
    #                   (total_time * cost_per_minute[vehicle_index]))

    #     return total_cost, total_distance, total_time
    def calculate_route_cost(route, vehicle_index, distance_matrix, time_matrices):
        total_distance = 0
        total_time = timedelta()  

        time_matrix = time_matrices[vehicle_index]
        depot_index = 0

        if route:
            travel_time_seconds = time_matrix[depot_index][route[0]]
            if isinstance(travel_time_seconds, list):
                raise TypeError(f"Expected a scalar value but got {type(travel_time_seconds)}. Check time_matrices structure.")
            total_distance += distance_matrix[depot_index][route[0]]  
            total_time += timedelta(seconds=travel_time_seconds)
            
            # Add serve time for the first stop
            total_time += timedelta(seconds=serve_time[route[0]])

        for i in range(len(route) - 1):
            total_distance += distance_matrix[route[i]][route[i + 1]]
            total_time += timedelta(seconds=time_matrix[route[i]][route[i + 1]])
            
            # Add serve time for each intermediate stop
            total_time += timedelta(seconds=serve_time[route[i + 1]])

        # Last leg (return to depot)
        if route:
            total_distance += distance_matrix[route[-1]][depot_index]  
            total_time += timedelta(seconds=time_matrix[route[-1]][depot_index])
            # Optionally add serve time for the last stop before returning to depot
            total_time += timedelta(seconds=serve_time[route[-1]])

        # Calculate excess distance cost
        excess_distance = max(0, total_distance - contract_distance[vehicle_index])
        # excess_time=max(0,total_time - contract_time[vehicle_index])
 
        variable_time_cost = total_time * cost_per_minute[vehicle_index]
        # if excess_time>0:
        #     variable_time_cost += excess_time* variable_cost_per_minute[vehicle_index]


        variable_distance_cost = total_distance * cost_per_km[vehicle_index]
        if excess_distance > 0:
            variable_distance_cost += excess_distance * cost_per_km[vehicle_index]

        # Calculate total cost
        total_cost = (fixed_cost[vehicle_index] +
                    variable_distance_cost +
                    (total_time.total_seconds() * cost_per_minute[vehicle_index]))

        return total_cost, total_distance, total_time.total_seconds()
    # from datetime import timedelta

    # def calculate_route_cost(route, vehicle_index, distance_matrix, time_matrices):
    #     total_distance = 0
    #     total_time = timedelta()  

    #     time_matrix = time_matrices[vehicle_index]
    #     depot_index = 0

        
    #     if route:
    #         travel_time_seconds = time_matrix[depot_index][route[0]]
    #         if isinstance(travel_time_seconds, list):
    #             raise TypeError(f"Expected a scalar value but got {type(travel_time_seconds)}. Check time_matrices structure.")
    #         total_distance += distance_matrix[depot_index][route[0]]  
    #         total_time += timedelta(seconds=travel_time_seconds)

        
    #     for i in range(len(route) - 1):
    #         total_distance += distance_matrix[route[i]][route[i + 1]]
    #         total_time += timedelta(seconds=time_matrix[route[i]][route[i + 1]])

    #     # Last leg
    #     if route:
    #         total_distance += distance_matrix[route[-1]][depot_index]  
    #         total_time += timedelta(seconds=time_matrix[route[-1]][depot_index])

    #     # Calculate excess distance cost
    #     excess_distance = max(0, total_distance - contract_distance[vehicle_index])
    #     variable_distance_cost = total_distance * cost_per_km[vehicle_index]
    #     if excess_distance > 0:
    #         variable_distance_cost += excess_distance * cost_per_km[vehicle_index]

    #     # Calculate total cost
    #     total_cost = (fixed_cost[vehicle_index] +
    #                 variable_distance_cost +
    #                 (total_time.total_seconds() * cost_per_minute[vehicle_index]))

    #     return total_cost, total_distance, total_time.total_seconds()  



    def local_cheapest_insertion_with_paired_orders(
        distance_matrix, time_matrices, num_vehicles, demands, vehicle_capacities,
        order_volume, vehicle_max_available_time, serve_time, vehicle_max_weight,
        vehicle_volume, order_types, paired_orders, paired_pickup_orders,
        paired_dropoff_orders, vehicle_time_windows, order_time_windows,
        vehicle_max_travel_distance, vehicle_max_order):

        #
        routes = [[] for _ in range(num_vehicles)]
        skipped_orders = []
        assigned_orders = set()  # Track orders that have been assigned
        unvisited_customers = set(range(1, len(demands)))  # Assuming customer IDs start from 1
        
        # 
        paired_pickup_orders = {pair[0]: pair[1] for pair in paired_pickup_orders}
        paired_dropoff_orders = {pair[0]: pair[1] for pair in paired_dropoff_orders}
        pairing_pickup_to_dropoff = {pair[0]: pair[1] for pair in paired_orders}
        paring_dropoff={pair[0]: pair[1] for pair in pairing_dropoff}

        def assign_orders_to_vehicle(v, current_node, unvisited_customers, skipped_orders):
            vehicle_capacity = vehicle_capacities[v]
            vehicle_start_time = vehicle_time_windows[v][0]
            vehicle_end_time = vehicle_time_windows[v][1]
            vehicle_max_time = vehicle_max_available_time[v]
            current_time = vehicle_start_time
            total_time_spent = 0
            current_route = []
            pickup_weight = 0
            dropoff_weight = 0
            available_weight = vehicle_capacity
            c=0
            

            while unvisited_customers:
                nearest_customer = None
                min_distance = float('inf')

                # Find the nearest unvisited customer
                for customer in unvisited_customers:
                    if distance_matrix[current_node][customer] < min_distance:
                        nearest_customer = customer
                        min_distance = distance_matrix[current_node][customer]

                if nearest_customer is None:
                    break  # No more customers

                order = nearest_customer
                ot = order_types[order - 1]  # Adjust zero-based indexing due to  input data 
                weight = demands[order]

                if order in assigned_orders:
                    unvisited_customers.remove(order)
                    continue

                # Ctime windows and arrival time for the order
                order_start_time = order_time_windows[order - 1][0]
                order_end_time = order_time_windows[order - 1][1]
                travel_time_seconds = time_matrices[v][current_node][order]
                travel_time = timedelta(seconds=travel_time_seconds)
                arrival_time = current_time + travel_time 
                departure_time = arrival_time + timedelta(seconds=serve_time[order - 1])
                remaining_vehicle_time = vehicle_max_available_time[v] 
                
                # c+= serve_time[order-1] + travel_time_seconds
                
                

                if not (vehicle_start_time <= order_end_time ):
                    skipped_orders.append(order)
                    unvisited_customers.remove(order)
                    continue 
                
                print((departure_time - vehicle_start_time).total_seconds())

                if (departure_time - vehicle_start_time).total_seconds() > vehicle_max_time:

                    skipped_orders.append(order)
                    # print(skipped_orders)
                    unvisited_customers.remove(order)
                    continue

                # print("pickup_weight",pickup_weight)
                # print("dropoff_weight",dropoff_weight)
                
                current_time = departure_time

                if ot == "pickup":
                    # Handle paired pickups
                    if order in paired_pickup_orders:
                        paired_pickup = paired_pickup_orders[order]
                        paired_weight = demands[paired_pickup]

                        if paired_pickup not in unvisited_customers:
                            skipped_orders.extend([order, paired_pickup])
                            unvisited_customers.remove(order)
                            continue

                        total_pickup_weight = weight + paired_weight
                        if pickup_weight + total_pickup_weight <= vehicle_capacity and available_weight >= total_pickup_weight :
                            current_route.extend([order, paired_pickup])
                            pickup_weight += total_pickup_weight
                            available_weight -= total_pickup_weight
                            assigned_orders.update([order, paired_pickup])
                            unvisited_customers.remove(order)
                            unvisited_customers.remove(paired_pickup)
                            current_node = paired_pickup
                            current_time = departure_time  # Update current time after servicing
                        else:
                            skipped_orders.extend([order, paired_pickup])
                            unvisited_customers.remove(order)
                            continue

                    # Handle paired pickup-dropoff
                    elif order in pairing_pickup_to_dropoff:
                        paired_dropoff = pairing_pickup_to_dropoff[order]
                        paired_dropoff_weight = demands[paired_dropoff]

                        if paired_dropoff not in unvisited_customers:
                            skipped_orders.extend([order, paired_dropoff])
                            unvisited_customers.remove(order)
                            continue

                        if pickup_weight + weight + paired_dropoff_weight <= vehicle_capacity and available_weight >= (weight + paired_dropoff_weight):
                            current_route.extend([order, paired_dropoff])
                            assigned_orders.update([order, paired_dropoff])
                            unvisited_customers.remove(order)
                            unvisited_customers.remove(paired_dropoff)
                            current_node = paired_dropoff
                            current_time = departure_time  # Update current time after servicing
                        else:
                            skipped_orders.extend([order, paired_dropoff])
                            unvisited_customers.remove(order)
                            continue

                    # Handle unpaired pickups
                    else:
                        if pickup_weight + weight <= vehicle_capacity and available_weight >= weight and departure_time < order_end_time:
                            current_route.append(order)
                            assigned_orders.add(order)
                            pickup_weight += weight
                            available_weight -= weight
                            unvisited_customers.remove(order)
                            current_node = order
                            current_time = departure_time  # Update current time after servicing
                        else:
                            skipped_orders.append(order)
                            unvisited_customers.remove(order)
                            continue

                elif ot == "dropoff":
                    # Handle paired dropoffs
                    if order in paired_dropoff_orders:
                        paired_dropoff = paired_dropoff_orders[order]
                        paired_weight = demands[paired_dropoff]

                        if paired_dropoff not in unvisited_customers:
                            skipped_orders.extend([order, paired_dropoff])
                            unvisited_customers.remove(order)
                            continue

                        total_dropoff_weight = weight + paired_weight
                        if dropoff_weight + total_dropoff_weight <= vehicle_capacity and available_weight >= total_dropoff_weight:
                            current_route.extend([order, paired_dropoff])
                            dropoff_weight += total_dropoff_weight
                            assigned_orders.update([order, paired_dropoff])
                            unvisited_customers.remove(order)
                            unvisited_customers.remove(paired_dropoff)
                            current_node = paired_dropoff
                            current_time = departure_time  # Update current time after servicing
                        else:
                            skipped_orders.extend([order, paired_dropoff])
                            unvisited_customers.remove(order)
                            continue

                    # Handle paired pickup-dropoff as a dropoff
                    elif order in pairing_dropoff:
                        paired_dropoffs = pairing_dropoff[order]
                        paired_dropoffs_weight = demands[paired_dropoffs]

                        if paired_dropoffs not in unvisited_customers:
                            skipped_orders.extend([order, paired_dropoffs])
                            unvisited_customers.remove(order)
                            continue

                        if dropoff_weight + weight + paired_dropoff_weight <= vehicle_capacity and available_weight >= (weight + paired_dropoff_weight):
                            current_route.extend([order, paired_dropoffs])
                            assigned_orders.update([order, paired_dropoffs])
                            unvisited_customers.remove(order)
                            unvisited_customers.remove(paired_dropoffs)
                            current_node = paired_dropoffs
                            current_time = departure_time  # Update current time after servicing
                        else:
                            skipped_orders.extend([order, paired_dropoffs])
                            unvisited_customers.remove(order)
                            continue

                    # Handle unpaired dropoffs
                    else:
                        if dropoff_weight + weight <= vehicle_capacity and available_weight > weight:
                            current_route.append(order)
                            assigned_orders.add(order)
                            dropoff_weight += weight
                            unvisited_customers.remove(order)
                            current_node = order
                            current_time = departure_time  # Update current time after servicing
                        else:
                            skipped_orders.append(order)
                            unvisited_customers.remove(order)
                            continue

            if current_route:
                routes[v] = current_route

        # Assign orders to each vehicle
        for v in range(num_vehicles):
            if not unvisited_customers:
                break  # All customers have been assigned
            assign_orders_to_vehicle(v, 0, unvisited_customers, skipped_orders)

        # Reassign skipped orders to the remaining vehicles in a round-robin manner
        unused_vehicles = [v for v in range(num_vehicles) if not routes[v]]
        for v in unused_vehicles:
            if not skipped_orders:
                break  # No skipped orders left
            assign_orders_to_vehicle(v, 0, set(skipped_orders), skipped_orders)

        skipped_orders = list(set(skipped_orders) - assigned_orders)
        return routes, skipped_orders




    # final_routes = routes_corrected
    final_routes = local_cheapest_insertion_with_paired_orders(distance_matrix, time_matrices, num_vehicles, demands, vehicle_capacities, order_volume, vehicle_max_available_time, serve_time, vehicle_max_weight, vehicle_volume, order_types, paired_orders,paired_pickup_orders,paired_dropoff_orders,vehicle_time_windows, order_time_windows,vehicle_max_travel_distance,vehicle_max_order)
    print(final_routes[0])


    csv_data = [["Vehicle", "Route", "Route Total Orders", "Dropoff Orders", "Pickup Orders", "Route Total Weight (kg)", "Route Total Volume (cm³)", "Route Cost", "Vehicle Fixed Cost", "Route Distance Cost", "Route Time Cost", "Route Distance (km)", "Vehicle Contract Distance (km)", "Detour Distance (km)", "Vehicle Cost per Unit Distance", "Vehicle Variable Cost per Unit Distance", "Route Time (s)", "Vehicle Contract Time (s)", "Overtime (s)", "Vehicle Cost per Unit Time", "Vehicle Variable Cost per Unit Time", "Vehicle Max Travel Distance (km)", "Vehicle Max Available Time (s)", "Vehicle Max Weight (kg)", "Vehicle Max Volume (cm³)", "Vehicle Distance Utilization (%)", "Vehicle Time Utilization (%)", "Vehicle Weight Utilization (%)", "Vehicle Volume Utilization (%)"]]

    total_cost = 0
    total_distance = 0
    total_time = 0 

    for i, route in enumerate(final_routes[0]):
        if len(route) > 0:
            route_cost, route_distance, route_time = calculate_route_cost(route, i, distance_matrix, time_matrices)
            

            
            route_total_orders = sum(1 for j in route if j != 0)  # Exclude depot

            pickup_orders = sum(1 for j in route if order_types[j - 1] == 'pickup') if len(route) > 0 else 0
            dropoff_orders = sum(1 for j in route if order_types[j - 1] == 'dropoff') if len(route) > 0 else 0
            
            route_total_weight = sum(demands[j] for j in route if j != 0)
            route_total_volume = sum(order_volume[j] for j in route if j != 0)
            
            vehicle_distance_utilization = (route_distance / vehicle_max_travel_distance[i]) * 100
            vehicle_time_utilization = (route_time / vehicle_max_available_time[i]) * 100
            vehicle_weight_utilization = (route_total_weight / vehicle_capacities[i]) * 100
            vehicle_volume_utilization = (route_total_volume / vehicle_volume[i]) * 100
            
            detour_distance = max(0, route_distance - contract_distance[i])
            overtime = max(0, route_time - contract_time[i])
            
            csv_data.append([
                vehicle_contract_code[i],
                " -> ".join(map(str, route)),
                route_total_orders,
                dropoff_orders,
                pickup_orders,
                route_total_weight,
                route_total_volume,
                route_cost,
                fixed_cost[i],
                cost_per_km[i]*route_distance,
                (route_time * cost_per_minute[i]),
                route_distance,
                contract_distance[i],
                detour_distance,
                cost_per_km[i],
                cost_per_km[i],
                route_time,
                contract_time[i],
                overtime,
                cost_per_minute[i],
                variable_cost_per_minute[i],
                vehicle_max_travel_distance[i],
                vehicle_max_available_time[i],
                vehicle_max_weight[i],
                vehicle_volume[i],
                vehicle_distance_utilization,
                vehicle_time_utilization,
                vehicle_weight_utilization,
                vehicle_volume_utilization
            ])

            total_cost += route_cost
            total_distance += route_distance
            total_time += route_time


    # Save the CSV results in a specific folder
    output_dir = os.path.join(result_folder, os.path.basename(file_path).replace('.json', ''))
    os.makedirs(output_dir, exist_ok=True)

    csv_file_path = os.path.join(output_dir, 'results.csv')
    with open(csv_file_path, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerows(csv_data)

    end_time = time.time()


    print(f"Processed file: {file_path}")
    print(f"Total cost: {total_cost}")
    print(f"Total distance: {total_distance}")
    print(f"Total time: {total_time}")
    print(f"Time taken: {end_time - start_time} seconds")
    print(f"Results saved to: {csv_file_path}")
 
    end_time = time.time()
    duration = end_time - start_time

    # Prepare result output
    result = {
        "routes": final_routes,
        "duration": duration
    }

    result_file = os.path.join(result_folder, "result.json")
    with open(result_file, 'w') as f:
        json.dump(result, f, indent=4)

    print(f"Results saved to {result_file}")

if __name__ == "__main__":
    input_file = 'data1.json'  # Replace with your input file path
    result_folder = '/home/delhivery/Desktop/vrp_project/data'  # Replace with your result folder path

    if not os.path.exists(result_folder):
        os.makedirs(result_folder)

    process_vrp_file(input_file, result_folder)
    

[36000, 36000, 864000]
2380.32
4494.24
6706.594286
8533.337143
10424.468572
12224.468572
14024.468572
16461.668572
18807.634286
21260.16
23447.211429
25464.445715
27996.582858
31413.497144
33255.05143
33261.942859
35144.125716
37042.045716
37042.354287
37090.285716
37394.742859
37800.925716
38139.017145
38145.908573
38252.365716
39011.040002
39167.382859
39207.805716
39302.33143
39648.65143
2921.862857
5450.708571
8026.457142
9836.537142
11636.845713
13510.799999
15692.091428
18243.257142
20463.839999
23864.70857
26097.839999
27975.702856
29987.279999
32371.405713
[[10, 20, 30, 13, 17, 25, 26, 21, 29, 11, 28, 18, 22, 5, 24, 1], [12, 14, 9, 8, 23, 3, 15, 4, 16, 7, 27, 19, 6, 2], []]
Processed file: data1.json
Total cost: 13558.896461623932
Total distance: 178.247
Total time: 75933.977144
Time taken: 0.0011801719665527344 seconds
Results saved to: /home/delhivery/Desktop/vrp_project/data/data1/results.csv
Results saved to /home/delhivery/Desktop/vrp_project/data/result.json


In [2]:
from datetime import datetime,timedelta